#### Simulation setup
- **Step 1:** Propagation of a Gaussian laser pulse in vacuum
    - Observe a laser pulse that propgates through a focus in vacuum
- **Step 2:** Excitation of a plasma wave
    - Observe a linear plasma wave with a0 = 1
    - Task: Change a0 to 2 to observe a non-linear plasma wave
- **Step 3:** Localized ionization injection of an electron beam
    - Add local dopant (nitrogen) gas and inject an electron bunch via ionization injection
    - Task: Tune the longitudinal focus position to optimize beam loading & minimize the energy spread 

### Installation

In [ ]:
# !pip install fbpic

### Package imports

- Explain additional software packages that are used
- Explain FBPIC imports

In [ ]:
import numpy as np                              # Array creation & manipulation
from scipy.constants import c, e, m_e, m_p, pi  # Physical constants
import numexpr as ne                            # Evaluate numerical expressions

In [ ]:
# Import FBPIC classes & functions
from fbpic.main import Simulation
from fbpic.lpa_utils.laser import add_laser_pulse
from fbpic.lpa_utils.laser.laser_profiles import GaussianLaser
from fbpic.lpa_utils.boosted_frame import BoostConverter
from fbpic.openpmd_diag import BackTransformedFieldDiagnostic, BackTransformedParticleDiagnostic

### Lorentz-Boosted Frame

- Discuss Boosted frame concept

In [ ]:
# Define the Lorentz factor of the Boosted frame
gamma_boost = 4.
# Create a boosted frame converter
boost = BoostConverter(gamma_boost)

### Simulation Box

- Explain box boundaries & mode decomposition
- Show schematic of the grid
- Explain that we use 3 number of modes

In [ ]:
# Simulation box
Nz = 1500        # Number of gridpoints along z
zmax = 0.e-6     # Right boundary of the box along z (meters)
zmin = -30.e-6   # Left boundary of the box along z (meters)
Nr = 150         # Number of gridpoints along r
rmax = 30.e-6    # Length of the box along r (meters)
Nm = 3           # Number of azimuthal modes used

# Simulation timestep
dt = min( rmax/(2*boost.gamma0*Nr)/c, (zmax-zmin)/Nz/c )  # Timestep (seconds)

### Simulation

- Explain simulation object and numerical parameters
- Boundary conditions, Macroparticle shape, Galilean Frame, ...

In [ ]:
# Simulation parameters
sim_params = {
    'Nz': Nz, 'zmin': zmin, 'zmax': zmax,            # Box parameters
    'Nr': Nr, 'rmax': rmax,                          # Box parameters
    'Nm': Nm, 'dt': dt,                              # Box parameters
    'boundaries': {'z':'open', 'r':'open'},          # Boundary conditions
    'particle_shape': 'cubic',                       # Marcoparticle shape factor
    'gamma_boost': boost.gamma0,                     # Lorentz factor of the Boosted frame
    'v_comoving': -c*np.sqrt(1.-1./boost.gamma0**2), # Velocity of the Galilean frame (NCI suppression)
    'use_cuda': True,                                # Simulate on CUDA-enabled GPU
}

#### Initialization
*Start from here when you want to re-run the simulation*

In [ ]:
# Initialize the simulation
sim = Simulation(**sim_params)

### Moving window & interaction time

- Explain moving window concept

In [ ]:
# Moving window speed
v_window = c
# The interaction length of the simulation, in the lab frame (meters)
L_interact = 700.e-6
# Interaction time, in the lab frame (seconds)
T_interact = (L_interact + (zmax-zmin)) / v_window 

In [ ]:
# Add the moving window to the simulation
sim.set_moving_window( v = boost.velocity([v_window])[0] )

### Laser

- Explain Gaussian laser parameters
- Explain concept of creating and adding a laser profile
- Explain working principle of laser antenna

In [ ]:
# Laser parameters
laser_params = {
    'a0': 2.,           # Laser amplitude 
    'waist': 10.e-6,    # Laser waist (meters)
    'tau': 12.e-15,     # Laser duration (seconds)
    'z0': -8.e-6,      # Laser centroid (meters)
    'zf': 290.e-6,        # Focal position (meters)
    'lambda0': 800.e-9  # Laser wavelength (meters)
}

# Create the laser profile
laser_profile = GaussianLaser(**laser_params)

In [ ]:
# Add the laser to the simulation
add_laser_pulse( sim, laser_profile, gamma_boost=boost.gamma0, method='antenna', z0_antenna=0)

### Gas & plasma density profile (only in step 2)

- Explain the density function
- Explain how to define a density profile based on conditional expressions

In [ ]:
# Gas density
n0 = 4.e24 # Gas atoms per unit volume [1/m^3]

# Gas profile
ramp_up = 100.e-6    # Length of density up-ramp (rising half-cosine)
plateau = 500.e-6   # Length of density plateau
ramp_down = 100.e-6  # Length of density down-ramp (falling half-cosine)

# Gas density (Hydrogen gas is assumed)
def hydrogen_density(z, r):
    # Conditional expression describing the profile
    profile = "(z>0) * n0*0.5*(1-cos(pi*(z)/ramp_up)) * (z<ramp_up) + " + \
              "(z>=ramp_up) * n0 * (z<=ramp_up+plateau) + " + \
              "(z>ramp_up+plateau) * n0*0.5*(1+cos(pi*(z-(ramp_up+plateau))/ramp_down)) * (z<ramp_up+plateau+ramp_down)"
    # Evaluate expression
    n = ne.evaluate(profile)
    return n

# Plasma electron density (assuming full gas ionization)
def electron_density(z, r):
    # Simply inferred from the Hydrogen gas profile
    n = hydrogen_density(z, r) # One electron per Hydrogen ion
    return n

#### Dopant gas for ionization injection (only in step 3)

- Explain Ionization injection
- Localized dopant gas profile
- Explain that dopant gas is pre-ionized for performance reasons

In [ ]:
# Dopant gas density fraction
f = 0.05

# Dopant gas density profile (Nitrogen gas is assumed)
def nitrogen_density(z, r):
    # Conditional expression describing the profile
    profile = "(z>0) * f*n0*0.5*(1-cos(pi*(z)/ramp_up)) * (z<ramp_up) + " + \
              "(z>ramp_up) * f*n0*0.5*(1+cos(pi*(z-(ramp_up))/ramp_down)) * (z<ramp_up+ramp_down)"
    # Evaluate expression
    n = ne.evaluate(profile)
    return n

# Plasma electron density (assuming full gas ionization)
def electron_density(z, r):
    # Simply inferred from the Hydrogen & preionized (up to 5th level) Nitrogen gas profile
    n = hydrogen_density(z, r) + 5 * nitrogen_density(z, r) # 1 e- per H-ion & 5 e- per N-ion
    return n

### Particles (only in step 2)

- Explain Macroparticle parameters
- Particles per cell in each dimension
- Explain different species

In [ ]:
# Macroparticle parameters
part_params = {
    'p_zmin': 0.e-6,   # Start of particles (meters)
    'p_zmax': 700.e-6, # End of particles (meters)
    'p_rmax': 30.e-6,  # Radial limit of particles (meters)
    'p_nz': 1,         # Number of particles per cell along z
    'p_nr': 2,         # Number of particles per cell along r
    'p_nt': 6          # Number of particles per cell along theta
} 

In [ ]:
# Add the plasma electrons
electrons = sim.add_new_species( q=-e, m=m_e, n=1, dens_func=electron_density,
                                 **part_params, boost_positions_in_dens_func=True )
# Add the hydrogen ions
hydrogen_ions = sim.add_new_species( q=e, m=m_p, n=1, dens_func=hydrogen_density,
                                     **part_params, boost_positions_in_dens_func=True )

#### Ionization (only in step 3)

- Explain ionization feature and ADK model
- Explain that its useful to add an additional species just for the ionized electrons

In [ ]:
# Add the nitrogen ions (assuming pre-ionization up to the 5th level)
nitrogen_ions = sim.add_new_species( q=5*e, m=14.*m_p, n=1, dens_func=nitrogen_density,
                                     **part_params, boost_positions_in_dens_func=True )
# Add a second electron species for storing the ionized electrons separately
ionized_electrons = sim.add_new_species( q=-e, m=m_e )

In [ ]:
# Make Nitrogen atoms ionizable (assuming pre-ionization up to the 5th level)
nitrogen_ions.make_ionizable( 'N', target_species=ionized_electrons, level_start=5 )

### Diagnostics

- Explain general functionality of diagnostics
- Explain the caveats of back-transforming the results from the boosted frame to the lab frame

In [ ]:
# Remove diagnostic directory
!rm -r ./lab_diags

In [ ]:
# Number of discrete diagnostic snapshots, back-transformed from the boosted simulation frame to the lab frame
N_diag = 10+1
# Time interval between diagnostic snapshots *in the lab frame* 
# (first at t=0, the last at t=T_interact)
dt_diag_period = T_interact / (N_diag - 1) 

#### Field diagnostics

In [ ]:
# Field diagnostics (back-transformed to lab frame)
field_diag = BackTransformedFieldDiagnostic( zmin, zmax, v_window, dt_diag_period, N_diag, boost.gamma0,
                                             period=50, fldobject=sim.fld, comm=sim.comm, 
                                             fieldtypes=['rho','E','B'] )
# Add the diagnostic to the simulation
sim.diags.append( field_diag )

#### Particle diagnostics (only in step 3)

In [ ]:
# Particle diagnostics (back-transformed to lab frame)
particle_diag = BackTransformedParticleDiagnostic( zmin, zmax, v_window, dt_diag_period, N_diag, boost.gamma0,
                                                   period=50, fldobject=sim.fld, comm=sim.comm, 
                                                   species={'bunch': ionized_electrons} )
# Add the diagnostic to the simulation
sim.diags.append( particle_diag )

### PIC Loop

In [ ]:
# Number of PIC loop iterations (Interaction time in the Boosted frame divided by the timestep)
N_step = int( boost.interaction_time( L_interact, (zmax-zmin), v_window ) / sim.dt )

# Run Particle-In-Cell loop
sim.step(N_step)